- Yolo는 80여가지의 사물을 미리 학습시켜놓음 -> 높은 인식률을 보임
- haar [x, y, w, h] # 좌측상단x, 좌측상단y, 가로길이, 세로길이
- dnn [x, y, x, y] # 좌측상단x 비율, 좌측상단y 비율, 우측하단 x비율, 우측하단 y비율 
- yolo [x, y, 가로비율, 세로비율]

In [1]:
import cv2
import numpy as np

In [2]:
# 최소 정확도
min_confidence = 0.5

In [3]:
# yolo 모델을 로딩한다
net = cv2.dnn.readNet('opencv_data/yolo/yolov3.weights', 'opencv_data/yolo/yolov3.cfg')

In [4]:
# 물체 종류를 리스트로 추출한다
with open('opencv_data/yolo/coco.names', 'r') as fp:
    classes = [line.strip() for line in fp.readlines()]
    
classes

['person',
 'bicycle',
 'car',
 'motorbike',
 'aeroplane',
 'bus',
 'train',
 'truck',
 'boat',
 'traffic light',
 'fire hydrant',
 'stop sign',
 'parking meter',
 'bench',
 'bird',
 'cat',
 'dog',
 'horse',
 'sheep',
 'cow',
 'elephant',
 'bear',
 'zebra',
 'giraffe',
 'backpack',
 'umbrella',
 'handbag',
 'tie',
 'suitcase',
 'frisbee',
 'skis',
 'snowboard',
 'sports ball',
 'kite',
 'baseball bat',
 'baseball glove',
 'skateboard',
 'surfboard',
 'tennis racket',
 'bottle',
 'wine glass',
 'cup',
 'fork',
 'knife',
 'spoon',
 'bowl',
 'banana',
 'apple',
 'sandwich',
 'orange',
 'broccoli',
 'carrot',
 'hot dog',
 'pizza',
 'donut',
 'cake',
 'chair',
 'sofa',
 'pottedplant',
 'bed',
 'diningtable',
 'toilet',
 'tvmonitor',
 'laptop',
 'mouse',
 'remote',
 'keyboard',
 'cell phone',
 'microwave',
 'oven',
 'toaster',
 'sink',
 'refrigerator',
 'book',
 'clock',
 'vase',
 'scissors',
 'teddy bear',
 'hair drier',
 'toothbrush']

In [5]:
# 은닉층의 이름들을 추출한다
layer_names = net.getLayerNames()
# 다수의 신경망을 사용하는 yolo 모델에서 각 신경망의 출력층 이름을 가져온다
output_layers = [layer_names[i[0] - 1] for i in net.getUnconnectedOutLayers()]

In [6]:
# 각 물체에 대해 표시할 영역의 색상을 랜덤으로 정해준다
colors = np.random.uniform(0, 255, size=(len(classes), 3))
colors

array([[ 96.89852745, 223.66935204, 173.52404204],
       [173.80985622, 166.00924378, 182.42965286],
       [ 32.28127873, 131.45450364, 174.96633665],
       [ 35.40447273,  47.66830751, 200.65698346],
       [222.09087551,   8.9343687 ,  26.08573805],
       [ 23.40093999, 201.76997387,  88.93135069],
       [ 93.26419244,  59.51587276, 229.89665618],
       [  6.58685416, 160.19732701, 128.47175859],
       [ 69.52983605, 211.61297212, 131.20522569],
       [126.22573578,  54.21844121, 156.5241478 ],
       [ 49.04810527, 251.50086321, 124.31390049],
       [222.54436057, 196.76740922,  12.4765026 ],
       [ 52.53651935,  11.52988272, 246.15186542],
       [203.21284001, 101.47286027,  41.37251809],
       [ 48.11309626, 109.60278867, 198.81419588],
       [250.42737661, 229.41280319, 178.60037451],
       [ 44.40124253, 201.91987794, 235.3752666 ],
       [ 86.00437306,  69.97297674,  58.82607873],
       [217.57349821, 240.37410076,   3.8201039 ],
       [ 37.26450215,  72.33330

In [7]:
# 이미지를 불러온다
img = cv2.imread('opencv_data/image/yolo_01.jpg')

In [8]:
# 이미지의 정보를 추출한다
height, width, channels = img.shape
# 원본 이미지를 띄운다
cv2.imshow('Original Image', img)

In [9]:
# 이미지를 리사이징해서 이진데이터로 만든다
# 320X320, 416X416, 609X609 => 이미지가 커지면 처리 속도가 느리지만 인식률이 좋음 => 그 중간이 416X416
blob = cv2.dnn.blobFromImage(img, 0.00392, (416, 416), (0, 0, 0), True, crop=False)

In [10]:
# 2진 데이터를 넣어서 예측 결과를 가져온다
net.setInput(blob)
outs = net.forward(output_layers)

In [11]:
# 예측된 결과를 담을 리스트
# 인지된 사물의 인덱스 번호
class_ids = []
# 예측 정확도
confidences = []
# 인지된 사물의 영역
boxes = []

In [12]:
# 넘파이 행렬의 경우 판다스처럼 value_counts가 없기 때문에 import collections를 해 줌!

import collections

# 각 신경망이 예측한 결과 묶음의 개수(3개) 만큼 반복한다.
for out in outs :
    # 예측된 사물의 수 만큼 반복한다.
    for detection in out :
        # 예측 정확도를 가지고 있는 배열을 추출한다.
        scores = detection[5:]
        # 가장 높은 정확도를 가지고 있는 곳의 인덱스를 추출한다.
        class_id = np.argmax(scores)
        #print(class_id)
        #print(collections.Counter(scores))
        
        # 예측 정확도를 가져온다
        confidence = scores[class_id]
        # print(confidence)
        
        # 예측 정확도가 설정한 최소 정확도 보다 큰 것만 사용한다
        if confidence > min_confidence:

            # 감지한 물체에 영역을 표시한다
            # 물체의 중앙점 x, y좌표
            center_x = int(detection[0] * width)
            center_y = int(detection[1] * height)

            # 물체의 가로, 세로 길이
            w = int(detection[2] * width)
            h = int(detection[3] * height)

            # 영역의 x, y좌표
            x = int(center_x - w / 2)
            y = int(center_y - h / 2)

            # print(x, y, w, h)

            cv2.rectangle(img, (x, y), (x+w, y+h), colors[class_id], 2)
            
            # 물체 이름을 표시한다
            text = f'{classes[class_id]} : {int(confidence * 100)}'
            cv2.putText(img, text, (x, y-5), cv2.FONT_HERSHEY_PLAIN, 1, colors[class_id], 1)
            
            # 검출된 항목의 인덱스를 담는다.
            class_ids.append(class_id)
            # 검출된 항목의 예측 정확도를 담는다.
            confidences.append(confidence)
            # 인지된 사물의 영역을 담는다.
            boxes.append((x, y, w, h))

In [13]:
# 결과 출력
for idx in range(len(class_ids)):
    a1 = classes[class_ids[idx]]
    a2 = confidences[idx]
    a3 = boxes[idx]
    
    print(f'이름 : {a1}')
    print(f'정확도 : {a2}')
    print(f'영역 : {a3}')
    print('---------------')

이름 : car
정확도 : 0.9808712005615234
영역 : (6, 364, 223, 115)
---------------
이름 : car
정확도 : 0.8883683681488037
영역 : (9, 371, 219, 117)
---------------
이름 : car
정확도 : 0.969380259513855
영역 : (329, 378, 203, 128)
---------------
이름 : car
정확도 : 0.9711914658546448
영역 : (8, 361, 217, 121)
---------------
이름 : car
정확도 : 0.942772388458252
영역 : (14, 359, 201, 122)
---------------
이름 : car
정확도 : 0.7079911828041077
영역 : (43, 364, 174, 110)
---------------
이름 : car
정확도 : 0.6161088347434998
영역 : (38, 357, 191, 123)
---------------
이름 : car
정확도 : 0.9755211472511292
영역 : (516, 386, 82, 68)
---------------
이름 : person
정확도 : 0.9571173191070557
영역 : (860, 369, 43, 112)
---------------
이름 : person
정확도 : 0.9603180289268494
영역 : (926, 376, 45, 96)
---------------
이름 : car
정확도 : 0.9945846796035767
영역 : (340, 388, 187, 108)
---------------
이름 : car
정확도 : 0.9956751465797424
영역 : (342, 374, 184, 136)
---------------
이름 : person
정확도 : 0.9896934628486633
영역 : (580, 380, 73, 127)
---------------
이름 : person
정확도 : 0.

In [14]:
cv2.imshow('YOLO Image', img)
cv2.waitKey(0)
cv2.destroyAllWindows()